In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, classification_report


In [2]:
# ================= CONFIG =================
DL_TENSOR_DIR = Path("data/dl_ready_trials")
ML_LABEL_FILE = r"data/clusters/features_master_labeled_gmm4.csv"

LABEL_COL = "performance_level"   # <<< CHANGE HERE IF NEEDED

BATCH_SIZE = 8
EPOCHS = 40
LR = 1e-3
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


In [3]:
df_ml = pd.read_csv(ML_LABEL_FILE)

# keep only required columns
CONF_COL = "performance_level_confidence"
CONF_THRESHOLD = 0.6   # recommended starting point

df = df_ml[['file', 'player', LABEL_COL, CONF_COL]].copy()

# drop tentative / invalid labels
# df = df[~df[LABEL_COL].astype(str).str.contains("TENTATIVE", case=False)]

# keep only confident samples
df = df[df[CONF_COL] >= CONF_THRESHOLD]

df = df.dropna(subset=[LABEL_COL]).reset_index(drop=True)

# drop tentative / invalid labels if present
# df = df[~df[LABEL_COL].astype(str).str.contains("TENTATIVE", case=False)]
# df = df.dropna(subset=[LABEL_COL]).reset_index(drop=True)

# map CSV file → tensor file
df['tensor_name'] = df['file'].apply(lambda x: Path(x).stem + ".npy")
df['tensor_path'] = df['tensor_name'].apply(lambda x: DL_TENSOR_DIR / x)

# sanity check
missing = df[~df['tensor_path'].apply(lambda p: p.exists())]
assert len(missing) == 0, f"Missing tensors for {len(missing)} trials!"

print("Total trials:", len(df))
print("Players:", df['player'].nunique())
print("Label distribution:")
print(df[LABEL_COL].value_counts())


Total trials: 210
Players: 42
Label distribution:
performance_level
Poor             60
Below-Average    60
Excellent        48
Average          42
Name: count, dtype: int64


In [4]:
le = LabelEncoder()
df['y'] = le.fit_transform(df[LABEL_COL])

print("Encoded classes:")
for i, c in enumerate(le.classes_):
    print(i, "→", c)


Encoded classes:
0 → Average
1 → Below-Average
2 → Excellent
3 → Poor


In [5]:
class EMGTensorDataset(Dataset):
    def __init__(self, df):
        self.df = df.reset_index(drop=True)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        X = np.load(row['tensor_path'])   # (channels, time)
        y = row['y']
        player = row['player']

        return (
            torch.tensor(X, dtype=torch.float32),
            torch.tensor(y, dtype=torch.long),
            player
        )


In [6]:
class EMGCNN(nn.Module):
    def __init__(self, n_channels, n_classes):
        super().__init__()

        self.net = nn.Sequential(
            nn.Conv1d(n_channels, 32, kernel_size=7, padding=3),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(2),

            nn.Conv1d(32, 64, kernel_size=5, padding=2),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(2),

            nn.Conv1d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),

            nn.AdaptiveAvgPool1d(1),
            nn.Flatten(),

            nn.Dropout(0.4),
            nn.Linear(128, n_classes)
        )

    def forward(self, x):
        return self.net(x)


In [7]:
players = df['player'].unique()

all_true = []
all_pred = []

for test_player in players:
    print(f"\nLOPO fold — Test player: {test_player}")

    train_df = df[df['player'] != test_player]
    test_df  = df[df['player'] == test_player]

    train_ds = EMGTensorDataset(train_df)
    test_ds  = EMGTensorDataset(test_df)

    train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
    test_loader  = DataLoader(test_ds, batch_size=1, shuffle=False)

    model = EMGCNN(
        n_channels=train_ds[0][0].shape[0],
        n_classes=len(le.classes_)
    ).to(DEVICE)

    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    criterion = nn.CrossEntropyLoss()

    # -------- TRAIN --------
    model.train()
    for _ in range(EPOCHS):
        for X, y, _ in train_loader:
            X, y = X.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad()
            loss = criterion(model(X), y)
            loss.backward()
            optimizer.step()

    # -------- TEST --------
    model.eval()
    with torch.no_grad():
        for X, y, _ in test_loader:
            X = X.to(DEVICE)
            pred = torch.argmax(model(X), dim=1).cpu().item()
            all_pred.append(pred)
            all_true.append(y.item())



LOPO fold — Test player: Ahesan

LOPO fold — Test player: Devansh

LOPO fold — Test player: Jordan

LOPO fold — Test player: Karan

LOPO fold — Test player: Nihaal

LOPO fold — Test player: Soham

LOPO fold — Test player: Yadnesh

LOPO fold — Test player: dummy01

LOPO fold — Test player: dummy02

LOPO fold — Test player: dummy03

LOPO fold — Test player: dummy04

LOPO fold — Test player: dummy05

LOPO fold — Test player: dummy06

LOPO fold — Test player: dummy07

LOPO fold — Test player: dummy08

LOPO fold — Test player: dummy09

LOPO fold — Test player: dummy10

LOPO fold — Test player: dummy11

LOPO fold — Test player: dummy12

LOPO fold — Test player: dummy13

LOPO fold — Test player: dummy14

LOPO fold — Test player: dummy15

LOPO fold — Test player: dummy16

LOPO fold — Test player: dummy17

LOPO fold — Test player: dummy18

LOPO fold — Test player: dummy19

LOPO fold — Test player: dummy20

LOPO fold — Test player: dummy21

LOPO fold — Test player: dummy22

LOPO fold — Test pla

In [8]:
print("\nDL LOPO Accuracy:", accuracy_score(all_true, all_pred))
print("DL LOPO Macro-F1:", f1_score(all_true, all_pred, average="macro"))
print(classification_report(all_true, all_pred, target_names=le.classes_))



DL LOPO Accuracy: 1.0
DL LOPO Macro-F1: 1.0
               precision    recall  f1-score   support

      Average       1.00      1.00      1.00        42
Below-Average       1.00      1.00      1.00        60
    Excellent       1.00      1.00      1.00        48
         Poor       1.00      1.00      1.00        60

     accuracy                           1.00       210
    macro avg       1.00      1.00      1.00       210
 weighted avg       1.00      1.00      1.00       210



In [9]:
print("Trials after confidence filtering:", len(df))
print("Players remaining:", df['player'].nunique())
print("Label distribution:")
print(df[LABEL_COL].value_counts())


Trials after confidence filtering: 210
Players remaining: 42
Label distribution:
performance_level
Poor             60
Below-Average    60
Excellent        48
Average          42
Name: count, dtype: int64
